Nabouwen voor BaseRNN model van Gestures datasets

Download data
Preprocess data

In [1]:
from mads_datasets import DatasetFactoryProvider, DatasetType
from mltrainer.preprocessors import PaddedPreprocessor
preprocessor = PaddedPreprocessor()

gesturesdatasetfactory = DatasetFactoryProvider.create_factory(DatasetType.GESTURES)
streamers = gesturesdatasetfactory.create_datastreamer(batchsize=32, preprocessor=preprocessor)
train = streamers["train"]
valid = streamers["valid"]

2024-12-05 15:48:12.757 | INFO     | mads_datasets.base:download_data:121 - Folder already exists at /home/azureuser/.cache/mads_datasets/gestures
100%|██████████| 651/651 [00:03<00:00, 181.23it/s]


In [2]:
from pathlib import Path
from mltrainer import TrainerSettings, ReportTypes
from mltrainer.metrics import Accuracy

accuracy = Accuracy()

settings = TrainerSettings(
    epochs=20,
    metrics=[accuracy],
    logdir=Path("gestures"),
    train_steps=len(train),
    valid_steps=len(valid),
    reporttypes=[ReportTypes.GIN, ReportTypes.TENSORBOARD, ReportTypes.MLFLOW],
    scheduler_kwargs={"factor": 0.5, "patience": 5},
    earlystop_kwargs=None
)
settings

2024-12-05 15:50:26.045 | INFO     | mltrainer.settings:check_path:61 - Created logdir /home/azureuser/MachineLearning/notebooks/3_recurrent_networks/gestures


epochs: 20
metrics: [Accuracy]
logdir: gestures
train_steps: 81
valid_steps: 20
reporttypes: [<ReportTypes.GIN: 1>, <ReportTypes.TENSORBOARD: 2>, <ReportTypes.MLFLOW: 3>]
optimizer_kwargs: {'lr': 0.001, 'weight_decay': 1e-05}
scheduler_kwargs: {'factor': 0.5, 'patience': 5}
earlystop_kwargs: None

In [3]:
len(train), len(valid)

(81, 20)

In [13]:
import gin
from mltrainer import rnn_models, Trainer

# Laad de configuratie
gin.parse_config_file("gestures.gin")

# Voorbeeld van het gebruik van de configuratie
model = rnn_models.BaseRNN()

# Voorbeeld van het gebruik van trainstreamer en validstreamer
trainstreamer = train.stream()
validstreamer = valid.stream()
x, y = next(iter(trainstreamer))
x.shape, y

(torch.Size([32, 39, 3]),
 tensor([10, 14, 12, 18,  7,  2, 18,  0, 17, 17, 14,  5,  6,  5, 12,  5,  5, 13,
          6,  0,  0,  2, 10, 14,  2, 17,  4,  3,  6,  2,  6,  6]))

Fill the gestures.gin file with relevant settings for `input_size`, `hidden_size`, `num_layers` and `horizon` (which, in our case, will be the number of classes...)

In [14]:
gin.get_bindings("BaseRNN")

{'input_size': 3, 'hidden_size': 64, 'num_layers': 5, 'horizon': 20}

Size model

In [15]:
yhat = model(x)
yhat.shape

torch.Size([32, 20])